In [24]:
from scipy.stats import wilcoxon
from scipy.stats import shapiro
import pandas as pd
import numpy as np
# import seaborn as sns
# import matplotlib.pyplot as plt
# import json

In [25]:
all= pd.read_csv('../AllWindowData.csv')
data=pd.DataFrame({'id':all["userId"],"case":all['caseNum'], 'Window Type':all['type'],'Environment': all[ "environment"],'Distance':all["distance"],'Window Anchor':all['windowMode'],'Opacity':all['transparency'],'Size':all['visual angle'],'Meeting Format':all['Meeting type'], 'Theta':all['theta'], 'Phi':all['phi'], 'Pitch':all['pitch'], 'Row':all['row'], 'Offset':all['offset'], 'x': all['x'], 'y': all['y'], 'z': all['z'], 'scale': all['scale']})
data['Window Type'] = data['Window Type'].replace({0: 'Shared Screen', 1: 'Speaker\'s Video'})
data['Window Anchor'] = data['Window Anchor'].replace({0: 'Path Anchor', 1: 'Head Anchor'})
data['Meeting Format'] = data['Meeting Format'].replace({1: 'Listening', 2: 'Discussion'})
data['Environment'] = data['Environment'].replace({'low': 'Low-Traffic', 'high': 'High-Traffic'})
data['Opacity']=1-data['Opacity']

In [26]:
def w(data, para, value):
    grouped = data.groupby(['Window Type', para]).apply(lambda x: x[[para, 'Window Type']+value])
    p = grouped[para].unique()
    title = f'{"Factor":11s}\t{"Window Type":15s}'
    for e in value:
        title += f'\t{e:>12s}'
    print(title)
    for w in ['Shared Screen', 'Speaker\'s Video']:        
        for pp in p:        
            
            a = grouped[(grouped['Window Type']==w) & (grouped[para]==pp)]
            
            mean = f'{pp:11s}\t{w:15s}'
            std = f'{pp:11s}\t{w:15s}'
            normal = f'{"Normal":11s}\t{w:15s}'
            for e in value:
                mean += f'\t{a[e].mean():12.2f}'
                std += f'\t{a[e].std():12.2f}'
                statistic_normal, p_value_normal = shapiro(a[e])
                if p_value_normal > 0.05:
                    normal += f'\t{"*":>12s}'
                else:
                    normal += f'\t{" ":>12s}'

            mean += "\tMean"
            std += "\tSTD"
            print(mean)
            print(std)
            print(normal)
    print("") 

In [27]:
w(data, 'Environment', ['Size', 'Distance', 'Theta', 'Phi',  'Opacity'])
w(data, 'Meeting Format', ['Size', 'Distance', 'Theta', 'Phi',  'Opacity'])
w(data, 'Window Anchor', ['Size', 'Distance', 'Theta', 'Phi',  'Opacity'])

Factor     	Window Type    	        Size	    Distance	       Theta	         Phi	     Opacity
Low-Traffic	Shared Screen  	      838.37	        0.86	       -5.88	      -18.77	        0.84	Mean
Low-Traffic	Shared Screen  	      426.52	        0.27	       15.64	       16.17	        0.14	STD
Normal     	Shared Screen  	            	            	           *	            	            
High-Traffic	Shared Screen  	      708.55	        0.88	       -4.72	      -12.35	        0.81	Mean
High-Traffic	Shared Screen  	      343.26	        0.29	       16.46	       20.55	        0.21	STD
Normal     	Shared Screen  	            	            	            	            	            
Low-Traffic	Speaker's Video	      129.94	        0.86	        7.50	      -16.23	        0.89	Mean
Low-Traffic	Speaker's Video	       74.87	        0.30	       22.59	       14.07	        0.17	STD
Normal     	Speaker's Video	            	            	           *	           *	            
High-Traffic	Speaker's Video	      136.50

In [7]:
def whp(data, para, value):
    print(f'{"Factor":11s}\t{"Window Anchor"}\t{"Window Type":15s}\t\t{"Value":12s}\t{"P-value":7s}\t{"sig":3s}\t{"mean1":7s}\t{"mean2":7s}')
    grouped = data.groupby(['Window Type', para]).apply(lambda x: x[[para, "Window Anchor", 'Window Type']+value])
    # print(grouped)
    p = grouped[para].unique()
    # All
    for anchor in ['Path Anchor', "Head Anchor"]: 
        a = grouped[(grouped[para]==p[0]) & (grouped['Window Anchor']==anchor)]
        b = grouped[(grouped[para]==p[1]) & (grouped['Window Anchor']==anchor)]
        for e in value:
            statistic, p_value = wilcoxon(np.array(a[e].tolist()), np.array(b[e].tolist()), alternative='two-sided')
            significant = getStar(p_value)
            print(f'{para:11s}\t{anchor:13s}\t{"All":15s}\t\t{e:12s}\t{p_value:.5f}\t{significant}\t{a[e].mean():.5f}\t{b[e].mean():.5f}')
    #
    for anchor in ['Path Anchor', "Head Anchor"]: 
        for w in ['Shared Screen', 'Speaker\'s Video']:                       
            a = grouped[(grouped['Window Type']==w) & (grouped['Window Anchor']==anchor) & (grouped[para]==p[0])]
            b = grouped[(grouped['Window Type']==w) & (grouped['Window Anchor']==anchor) & (grouped[para]==p[1])]
            for e in value:
                statistic, p_value = wilcoxon(np.array(a[e].tolist()), np.array(b[e].tolist()), alternative='two-sided')
                significant = getStar(p_value)
                print(f'{para:11s}\t{anchor:13s}\t{w:15s}\t\t{e:12s}\t{p_value:.4f}\t{significant:3s}\t{a[e].mean():.5f}\t{b[e].mean():.5f}')
    print("") 

In [8]:
whp(data, 'Environment', ['Size', 'Theta', 'Phi', 'Pitch', 'Row', 'Transparency', 'Distance'])
whp(data, 'Meeting Format', ['Size', 'Theta', 'Phi', 'Pitch', 'Row', 'Transparency', 'Distance'])

Factor     	Window Anchor	Window Type    		Value       	P-value	sig	mean1  	mean2  
Environment	Path Anchor  	All            		Size        	0.16553		0.14863	0.13342
Environment	Path Anchor  	All            		Theta       	0.48808		2.13435	-1.77294
Environment	Path Anchor  	All            		Phi         	0.00160	**	-16.05483	-8.79600
Environment	Path Anchor  	All            		Pitch       	0.07403		5.51962	3.42139
Environment	Path Anchor  	All            		Row         	0.89616		-3.63651	-3.32672
Environment	Path Anchor  	All            		Transparency	0.00272	**	0.87132	0.79579
Environment	Path Anchor  	All            		Distance    	0.26377		0.88760	0.85855
Environment	Head Anchor  	All            		Size        	0.05013		0.14633	0.12153
Environment	Head Anchor  	All            		Theta       	0.07590		-0.51103	1.31157
Environment	Head Anchor  	All            		Phi         	0.23726		-18.94667	-16.46174
Environment	Head Anchor  	All            		Pitch       	0.88023		6.13452	7.85329
Environmen

In [20]:
def wwt(data, value):
    print(f'{"Value":12s}\t{"P-value":7s}\t\t{"sig":3s}\t{"mean1":7s}\t{"mean2":7s}')
    grouped = data.groupby(['Window Type']).apply(lambda x: x[['Window Type']+value])

    # All
    a = grouped[(grouped['Window Type']=='Shared Screen')]
    b = grouped[(grouped['Window Type']=='Speaker\'s Video')]
    for e in value:
        statistic, p_value = wilcoxon(np.array(a[e].tolist()), np.array(b[e].tolist()), alternative='two-sided')
        significant = getStar(p_value)
        print(f'{e:12s}\t{p_value:.5f}\t\t{significant}\t{a[e].mean():.5f}\t{b[e].mean():.5f}')
    #
    # for w in ['Shared Screen', 'Speaker\'s Video']:                
    #     a = grouped[(grouped['Window Type']==w)]
    #     b = grouped[(grouped['Window Type']==w)]
    #     for e in value:
    #         statistic, p_value = wilcoxon(np.array(a[e].tolist()), np.array(b[e].tolist()), alternative='two-sided')
    #         significant = getStar(p_value)
    #         print(f'{w:15s}\t\t{e:12s}\t{p_value:.5f}\t{significant:3s}\t{a[e].mean():.5f}\t{b[e].mean():.5f}')
    print("") 

In [21]:
wwt(data, ['Size', 'Offset', 'Theta', 'Phi', 'Transparency'])

Value       	P-value		sig	mean1  	mean2  
Size        	0.00000		***	0.35421	0.19240
Offset      	0.00005		***	2.50064	2.37038
Theta       	0.00001		***	1.65888	1.47257
Phi         	0.07924			1.85226	1.81519
Transparency	0.49844			0.82223	0.81770

